In [1]:
import requests
from bs4 import BeautifulSoup
import lazynlp

def load_article(url):
    '''
    Loads an article and returns the title,text
    '''
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')

    title_html = soup.find_all('h1', class_='_23498')[0]
    title_txt = lazynlp.clean_html(str(title_html))
    content_html = soup.find_all('div', class_='_1_Akb')[0]
    content_txt = lazynlp.clean_html(str(content_html))

    return title_txt,content_txt
        
url = "https://timesofindia.indiatimes.com/india/single-day-jump-of-24248-covid-19-cases-pushes-indias-tally-close-to-7-lakh-mark/articleshow/76807460.cms"
article = load_article(url)[1]

In [3]:
import torch
from transformers import BertForQuestionAnswering
from transformers import BertTokenizer

#Model
model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

#Tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')



In [13]:
question = 'How many total deaths?'
paragraph = article

encoding = tokenizer.encode_plus(text=question,text_pair=paragraph)
inputs = encoding['input_ids']  #Token embeddings
sentence_embedding = encoding['token_type_ids']  #Segment embeddings
tokens = tokenizer.convert_ids_to_tokens(inputs) #input tokens

start_scores, end_scores = model(input_ids=torch.tensor([inputs]), token_type_ids=torch.tensor([sentence_embedding]))

start_index = torch.argmax(start_scores)

end_index = torch.argmax(end_scores)

answer = ' '.join(tokens[start_index:end_index+1])
corrected_answer = ''

for word in answer.split():
    
    #If it's a subword token
    if word[0:2] == '##':
        corrected_answer += word[2:]
    else:
        corrected_answer += ' ' + word

In [14]:
print(corrected_answer)
print(answer)

 19 , 693
19 , 69 ##3


In [2]:
print(article)

NEW DELHI: Another big single-day jump of 24,248  Covid-19 cases  took India's tally close to the 7-lakh mark on Monday, while the death toll due to the disease climbed to 19,693 with 425 new fatalities, according to the Union health ministry.  With this, the country has recorded over 20,000 cases of the infection for the fourth consecutive day.  India went past  Russia  on Sunday to become the third worst-hit nation by the Covid-19 pandemic. Only the US and  Brazil  are ahead of India in terms of total  coronavirus  infections.  According to the Monday morning update of the Union health ministry, a single-day jump of 24,248 Covid-19 cases pushed India's tally to 6,97,413.  With a steady rise, the number of recoveries stands at 4,24,432 in the country, while one patient has migrated. There are 2,53,287 active coronavirus cases in the country, the data showed.  "Thus, 60.85 per cent of the patients have recovered so far," the ministry said.  The total number of confirmed cases also incl